## TOC:
* [Binary Tree Nodes](#binary-tree-nodes)
* [Occupation - PIVOT](#occup)
* [THE PADS - CONCAT](#the_pads)
* [New Companies](#new_comps)

### Binary Tree Nodes<a class="anchor" id="binary-tree-nodes"></a>

- [Question Link in HackerRank](https://www.hackerrank.com/challenges/binary-search-tree-1?isFullScreen=true)
``` mysql
/*
    Logics:
    CASE WHEN <there is no parent> Then 'Root'
         WHEN <at least one node has this node as a parent> THEN 'Inner'
         ELSE <not a parent and no nodes have this as a parent> Then 'Leaf'
    END
*/
select n,
case 
when p is null then 'Root'
when n in (select distinct(p) from bst) then 'Inner'
else 'Leaf'
end
from bst
order by n;
```

### Occupations<a class="anchor" id="occup"></a>  
[Question Link in HackerRank](https://www.hackerrank.com/challenges/binary-search-tree-1?isFullScreen=true)

----------
```sql
-- MS SQL Server Solution
Select Doctor, Professor, Singer, Actor from (
 SELECT  name, 
               occupation,
        ROW_NUMBER() OVER (PARTITION BY occupation ORDER BY name) AS row_number
      FROM OCCUPATIONS
)dt
PIVOT (
    MAX(name)
    FOR [Occupation] IN ([Doctor],  [Professor],
                       [Singer],  [Actor])
)pt
ORDER BY row_number;
```

------
```sql
-- 2. MySQL Solution
-- There's no pivot function in MySQL, we can use CASE WHEN instead.
WITH ranking_table AS(
  SELECT name,
         occupation,
         DENSE_RANK() OVER (PARTITION BY occupation ORDER BY name) AS ranks
  FROM occupations
)
SELECT MAX(CASE WHEN occupation = 'Doctor' THEN name END) AS doctor,
       MAX(CASE WHEN occupation = 'Professor' THEN name END) AS professor,
       MAX(CASE WHEN occupation = 'Singer' THEN name END) AS singer,
       MAX(CASE WHEN occupation = 'Actor' THEN name END) AS actor
FROM ranking_table
GROUP BY ranks;
```

Notes 1: Differences between RANK() and DENSE_RANK()  
* `RANK() OVER (PARTITION BY ... ORDER BY ...)`: 1, 1, 3, ...
  * The use of the ORDER BY clause is necessary for using the RANK function.
  * PARTITION BY clause can be optional.
  * If two records share identical numerical values - they will also share a similar ranking value.
 
* `DENSE_RANK() OVER()`: 1, 1, 2, ...
  * It produces a Rank continuously without any gap.
  * Rows with identical values receive the same Rank.
  * The Rank of subsequent rows increases by one (within a PARTITION).

Notes 2: When do we use `MAX` with `CASE WHEN` in mysql ?
* The CASE WHEN spreads a column of data out across multiple columns making it diagonal
* The MAX compresses the diagonal data to one row, completing the rotation from vertical to horizontal
* Thease operations must be done in "vertical, diagonal, horizontal" order hence why the MAX(CASE WHEN..) so the case when is done first
----

### The Pads<a class="anchor" id="the_pads"></a>  

```sql
-- MySQL
SELECT CONCAT(Name, "(", LEFT(Occupation, 1), ")") name_prof
from Occupations
order by name_prof;

SELECT CONCAT('There are a total of ',COUNT(OCCUPATION),' ',LOWER(OCCUPATION),'s.')
FROM OCCUPATIONS
GROUP BY OCCUPATION
ORDER BY COUNT(OCCUPATION), OCCUPATION;
```

```sql
--- Sandbox
-- INIT database
CREATE TABLE OCCUPATIONS (
  Name VARCHAR(100),
  Occupation VARCHAR(255)
);

INSERT INTO Occupations(Name, Occupation) VALUES ('Julia', 'Actor');
INSERT INTO Occupations(Name, Occupation) VALUES ('Maria', 'Actor');
INSERT INTO Occupations(Name, Occupation) VALUES ('Ashley', 'Professor');
INSERT INTO Occupations(Name, Occupation) VALUES ('Belvet', 'Professor');
INSERT INTO Occupations(Name, Occupation) VALUES ('Britney', 'Professor');

-- QUERY database
SELECT Occupation, count(Occupation)
from OCCUPATIONS
Group BY Occupation
ORDER BY COUNT(OCCUPATION), OCCUPATION;

select concat("There", 1, "are") concat;

SELECT CONCAT('There are a total of ',COUNT(OCCUPATION),' ',LOWER(OCCUPATION),'s.') concat
FROM OCCUPATIONS
GROUP BY OCCUPATION
ORDER BY COUNT(OCCUPATION), OCCUPATION;
```

### New Companies<a class="anchor" id="new_comps"></a>  

```sql
--- Solution 1
SELECT COMPANY_CODE, FOUNDER,
(SELECT COUNT(DISTINCT LEAD_MANAGER_CODE) FROM LEAD_MANAGER WHERE COMPANY_CODE = C.COMPANY_CODE),
(SELECT COUNT(DISTINCT SENIOR_MANAGER_CODE) FROM SENIOR_MANAGER WHERE COMPANY_CODE = C.COMPANY_CODE),
(SELECT COUNT(DISTINCT MANAGER_CODE) FROM MANAGER WHERE COMPANY_CODE = C.COMPANY_CODE),
(SELECT COUNT(DISTINCT EMPLOYEE_CODE) FROM EMPLOYEE WHERE COMPANY_CODE = C.COMPANY_CODE)
FROM COMPANY C
ORDER BY COMPANY_CODE;
```

```sql
--- Solution 2
select c.company_code, c.founder,
       count(distinct l.lead_manager_code),
       count(distinct s.senior_manager_code),
       count(distinct m.manager_code),
       count(distinct e.employee_code)
from Company as c 
join Lead_Manager as l 
on c.company_code = l.company_code
join Senior_Manager as s
on l.lead_manager_code = s.lead_manager_code
join Manager as m 
on m.senior_manager_code = s.senior_manager_code
join Employee as e
on e.manager_code = m.manager_code
group by c.company_code, c.founder
order by c.company_code;
```